# Add transaction_value_SGD into MongoDB.Transaction

## Connect to database

In [1]:
from pymongo import MongoClient
import mysql.connector
from datetime import datetime


# MongoDB setup
mongo_client = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['Coinut']
collection = mongo_db['transaction']
# Fetch all doc data
data1 = collection.find()

# MySQL setup
mysql_db = mysql.connector.connect(
  user='root', password='liuwinq1', host='localhost', database='coinut')
mysql_cursor = mysql_db.cursor()

# Index Dictionary
index_dict = {'LTCBTC': 1, 'ETHBTC': 2, 'ETCBTC': 3, 'ETHLTC': 490569, 'ETCLTC': 490588, 'BTCUSDT': 490589, 'LTCUSDT': 490590, 'ETHUSDT': 490591, 'ETCUSDT': 490592, 'XMRUSDT': 852382, 'XMRBTC': 852383, 'XMRLTC': 852384, 'ZECUSDT': 1111976, 'ZECBTC': 1111977, 'ZECLTC': 1111978, 'BTCCAD': 1801843, 'LTCCAD': 1801844, 'ETHCAD': 1801845, 'ZECCAD': 1885998, 'BTCXSGD': 2372505, 'LTCXSGD': 2372506, 'ETHXSGD': 2372507, 'USDTXSGD': 2372508, 'DAIXSGD': 2372510, 'ETCXSGD': 2372511, 'BTCXIDR': 2372512}

In [ ]:
for transaction in data1:
    # Get instId and use it to find the currency pair
    inst_id = transaction['matchedOrder']['instId']
    pair = next((pair_name for pair_name, id_value in index_dict.items() if id_value == inst_id), None)
    
    if pair:
        # Split the currency pair
        split_idx = len(pair) // 2 if len(pair) % 2 == 0 else 3
        base_currency = pair[:split_idx]

    # Convert timestamp to datetime format
    timestamp = transaction['timestamp']
    dt = datetime.utcfromtimestamp(int(timestamp) / 1e6).strftime('%Y-%m-%d %H:%M:%S.%f')

    # Use timestamp to find the most recent price data for this base currency and SGD
    mysql_cursor.execute(f"SELECT `{base_currency}`, `SGD` FROM price WHERE time <= '{dt}' ORDER BY time DESC LIMIT 1")

    # Fetch the result
    result = mysql_cursor.fetchone()
    
   # If we got a result, calculate the value in SGD
    if result is not None:
        base_currency_price, sgd_price = result
        # Calculate the index price
        index_price = base_currency_price / sgd_price

        # Get the fill quantity
        fill_qty = float(transaction['fillQty'])

        # Calculate the transaction value in SGD
        transaction_value_sgd = fill_qty * float(index_price)

        # Add the transaction value to the transaction document
        transaction['transaction_value_sgd'] = transaction_value_sgd
        
        #Check out the value
        print(transaction_value_sgd)
        # Update the MongoDB document
        #collection.update_one({'_id': transaction['_id']}, {'$set': {'transaction_value_sgd': float(transaction_value_sgd)}})


0.09963009202277907
0.2959309664042943
0.2951418171605495
249.89051841984016
84.96277626274565
79.96496589434885
63.003664317508004
0.1000700140048236
0.09624481652232889
549.7591405236483
63.003664317508004
0.0499414266635493
0.0499414266635493
0.22422368333103718
0.04997810368396803
59.71445410495887
63.58319844910504
65.23126900460302
63.01447419696417
67.4325612012868
1422.8270952932003
0.453130627457959
66.29001625192157
68.6382661847756
0.19946551603340637
63.01447419696417
0.37339944601453673
0.19946551603340637
0.12985205093774754
0.021741741247641295
0.443212376626229
0.443212376626229
0.020544948151440857
0.495272876310948
0.49364377306488616
63.01447419696417
249.1155848719365
4.723741136917028
0.3499067508827428
0.6998135017654856
63.01447419696417
71.11236094815845
30.922769420223602
9.206978312541965
28.32863473136992
134.8651224025736
0.0015989608912049126
0.5700295577145513
0.10093440625731012
63.01447419696417
134.8651224025736
0.09853771816709397
0.943909663025331
134

55.340493363195066
11.585126177642014
408.79282575748596
314.3866906039364
738.2599869746241
29.840476518168824
28.670261752750438
66.70870340258156
6.6702241628847965
2.465016563643894
0.012544398966484517
0.004181466322161507
0.0001828308234515047
0.0001828308234515047
0.17644915837969916
121.15547347171062
59.797169851432834
117.92306459450096
9.57750778432495
279.20829568253316
120.73406312920032
70.29890713694515
0.27424623517725705
97.98748214142857
0.273514911883451
349.57903412786067
187.78140637336713
117.82250076276556
41.2891030608496
19.99757128303257
176.44915837969916
0.025596315283210654
23.644187222879687
0.019197236462407993
73.52048265820798
0.006216247997351159
0.007313232938060189
0.006216247997351159
0.006216247997351159
0.007066678084452945
990.4242066033262
59.039515019236646
31.29094296019542
71.49685268829558
0.0771421226780472
500.1827712429729
114.59569865233833
997.7466180901608
501.8358776635115
516.5957564183207
59.039515019236646
224.9405522232916
75.7476

105.71380161801058
169.19876559236545
650.4374657193949
167.7816905036522
275.38225962796497
64.62968065938382
0.45218114704349105
0.4496489326200475
0.0016278521293565677
0.0016278521293565677
0.0016278521293565677
0.0016927737333932344
51.060158389139495
102.22471340248362
102.34394907130456
0.2671897759672708
0.11253117297541992
0.11253117297541992
0.03750897914460033
0.03750897914460033
0.3635235047173752
102.41460724542067
616.2055203307219
198.96907460013298
0.0036352350471737524
0.2831093035002054
0.2827318244288718
0.2818174633064305
5.922751268823011
0.28152572680197246
0.016187506993055694
59.63801983433548
0.00018040420696800356
0.00018040420696800356
0.08743542597778768
2.915100238029294
0.00018026845242893514
0.10347409169420876
61.06433641104259
5.18012738988739
0.23316113594076715
0.020817958566139924
0.012490775139683954
1.9120721515826495
0.010735468949529477
1192.1284848291994
0.27860828030466905
250.61216003364106
9.475261653977373
40.35398023212994
0.453716580524528

134.64698341409033
134.64698341409033
134.64698341409033
134.64698341409033
134.64698341409033
134.64698341409033
195.2817111789621
49.110588677823316
12.277650535630412
3.0512756852417255
0.7667741764482203
0.1916969102866404
0.04792086139707474
0.2530079833138995
0.015054609921562613
65.00109146191225
65.41335233004787
294.1467704092949
3.289695098773055
297.1932361003482
93.5836255588035
46.86117994302305
0.28906942117052636
246.0152816785944
0.43596735462343605
0.6857127472763238
0.6087662107057521
0.000756231317646897
0.058796984947046244
232.3520723470091
60.55068537266939
71.10789624981356
8.004493870000843
16.235514916522707
38.9170487465571
11.079220077814915
0.005589734869452546
92.23062534596701
46.86117994302305
0.0007399659620504747
49.819383863213424
3.0307155890682322
7.926900745151218
53.23501859427715
0.0009315551547503841
0.0009315551547503841
88.7734953095252
85.70307423703534
2925.102681479864
0.005606700389362721
0.2822039195979236
0.02915484202468615
0.02693171261

291.0344725537083
0.0019334151730115807
0.29387910629776026
0.29349242326315794
0.00019334151730115805
0.292544916247461
0.2932990817458568
0.29155900809014634
107.89174402441266
0.2204129458533109
0.09033961667402937
0.21949601402626662
0.10680072614049096
0.158672869498996
0.291172325055544
0.29252571567665214
57.026558855618994
736.0832620407009
650.3205680751313
0.004156444577815474
0.0032117980828574116
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.00018892929899161245
0.0026450101858825742
0.002078222288907737
6.833297702758973
0.0026450101858825742
0.0026450101858825742
0.0026450101858825742
0.2596902163983939
0.009

...

28.985939097354198
29.075936042246866
28.937479203950453
29.075936042246866
28.937479203950453
29.075936042246866
29.048244674587583
29.075936042246866
28.937479203950453
27.691367659282733
27.623523808517486
27.691367659282733
27.594447872475243
29.075936042246866
28.991754284562646
0.0018495994521773363
0.09836796113747263
0.043475447225073884
139.39543177463128
2.0877540770574208
136.97428019512304
109.88979500397376
29.033141578875206
109.88979500397376
93.96276591229201
15.81714277626789
0.11899796080137069
0.12357480544757725
137.41643188806395
137.40256266186333
136.9784417246482
136.84072030847597
136.442534824256
104.27785757333633
0.04202375538789664
104.01919650469466
136.03769211145973
135.63992270402576
135.50233998011558
135.07059096849008
0.27167847646626353
48.225494479459414
48.07714042840108
48.029033116623964
47.871780603271596
47.823951368441165
47.705212512205165
47.65752231584808
47.539200575032105
47.491649417148366
47.27308093705118
47.22580785611413
47.11582842

31.88289615101126
31.789172361183947
31.757266390178906
31.66411234983383
31.632491253569903
31.539479650595386
31.50800099170202
31.44290711335691
31.411428454463543
31.316137853560093
31.284801632037283
31.189795905874952
31.1586021217227
31.064023707672042
0.04324057883777587
0.2189402399268527
0.04446871165460651
0.00039237098518770443
0.2189402399268527
0.09787476241626628
0.12499195717034986
1243.3107244994796
25.47893586030662
25.40172739622797
25.376324241688227
25.299401206312265
707.007223409685
684.0898634254817
692.020623601749
0.0014221578592730968
63.57532837388152
74.57525020990296
694.9659093984206
34.46101816697604
0.3573882700353292
75.01890385573046
75.01890385573046
4.650877709187178
0.29169267862497983
2.425484897110085
3.398100305102648
3.2043843732203117
3.438457790911468
0.008498047576269915
109.37650045686597
138.490394320099
0.007081706313558263
0.2832682525423305
0.0014009688289421216
99.73034773525411
99.35642915480946
99.25696036795458
98.88500314387045
98.

4813.259817626143
734.0963285629745
646.6975667274465
2.5631784203273082
0.007651278866648681
671.1528734248343
713.3709991085523
8.217483185212414
0.3416418372231627
0.33882831621073667
10.14476784627087
307.6518238492347
733.9726332014899
660.0908931362735
1.275750219708208
627.1516736470545
757.9621088131597
99.49474602206975
71.83842413001413
66.5660237273799
63.203430989048776
42.19584383554428
31.72206279560464
12.632173437028673
12.057947196111625
12.020969491376883
12.057947196111625
11.994174053163302
12.057947196111625
11.913787738522558
12.057947196111625
12.154812705253722
12.057947196111625
12.020969491376883
12.057947196111625
11.953980895842928
10.212813320724411
1.8451338753872144
12.087288200955495
12.057947196111625
12.017218130026981
12.057947196111625
12.011859042384264
12.057947196111625
12.021237445759018
12.057947196111625
12.01802199317339
12.057947196111625
12.0300799403695
12.02311312643397
11.987475193609905
11.976757018324474
11.80727587162357
11.79548587880

32.03900061370019
32.00696161308649
31.997570871527298
31.965669970054176
31.873281645008635
31.84138074353551
31.74899241848997
31.71722961615742
31.610893277913675
31.579268574721706
31.486604051395013
31.455117447343614
31.362314824876346
31.330966319965526
31.238025598357684
31.20681519258744
31.113736371839018
31.082525966068776
30.98944714532035
30.958512937831262
30.86515791880169
30.834361810453174
30.740868692283023
30.710072583934508
30.5889596376491
30.558301628441164
30.464670411130435
30.43428860020365
30.340381184611772
30.309999373684988
30.226863691058057
0.027481728974682488
0.4235500641474934
0.40977434290592024
0.41250617185862637
0.00956140133447147
0.011200498706095153
0.14888467792248436
0.26269709834672755
0.052539419669345507
0.052539419669345507
0.052539419669345507
0.052539419669345507
0.052539419669345507
0.26269709834672755
0.26269709834672755
747.450907553691
0.26269709834672755
637.2909911833436
0.26269709834672755
0.2735926696135194
0.2733194867182488
0.0

107.71782348537091
1.2697621796905987
134.4019368792229
107.71782348537091
0.2828866098432721
3.0284044867200306
0.1335144527308723
0.008638518369478671
0.007747550107155759
0.28350691208657813
0.12224481209277702
0.0013979636233008277
0.27721618650055413
79.18806883096237
78.9014862881857
78.28596290484636
78.00637018018618
78.28596290484636
78.10422763381723
78.20516060741956
77.98386296585103
77.90585659567085
77.63898533998272
76.88799928154552
76.66222815638244
2.767970742106381e-05
0.016132916769258612
78.24464142528413
77.9879990014092
78.14434602127535
77.8889839642601
77.81116611178805
77.5333265032361
76.82201158118805
76.60008132550907
76.82201158118805
76.53748561236884
0.2751366118481809
0.27470982289495205
0.3663271848547393
0.3608338714084676
0.36041002582733006
0.3587146435027797
69.11141276913303
0.35829079792164215
0.35574772443481656
0.3554651607140582
0.26888194615431027
0.26888194615431027
3.1600513711014866
72.96445166539446
75.78358990740095
0.5881563847585947
75

# Checkout

In [ ]:

dt = datetime.utcfromtimestamp(1641033236514062 / 1e6).strftime('%Y-%m-%d %H:%M:%S.%f')
print(dt)